Example 3: Full Pipeline Prediction

Input: A sequence of 10 windows of time series data
Process:

Each window is encoded by the VAE into the latent space
The sequence of 9 embeddings is fed into the LSTM
The LSTM predicts the 10th embedding
The predicted embedding is decoded by the VAE


Output: Predicted 10th window of the time series


Visualization:
The code includes visualization of the results:

In [1]:
%load_ext autoreload
%autoreload 2

In [42]:
import os
import argparse
import json
import torch
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import matplotlib.pyplot as plt

from arch_skip import VAE_LSTM_Model

def create_dirs(dirs):
    for dir_path in dirs:
        if not os.path.exists(dir_path):
            os.makedirs(dir_path)



In [4]:
class DataGenerator:
    def __init__(self):
        global train_configurations
        config = train_configurations
        self.config = config
        # Here you would load and preprocess your data
        # VAE
        self.train_data = torch.randn(1000, config['n_channel'], config['l_win'])
        self.val_data = torch.randn(200, config['n_channel'], config['l_win'])
        # LSTM
        self.train_set_lstm = {'data': torch.randn(100, config['l_seq'], config['l_win'], config['n_channel'])}
        self.val_set_lstm = {'data': torch.randn(20, config['l_seq'], config['l_win'], config['n_channel'])}
        # Set the number of training and validation samples
        self.n_train_lstm = self.train_set_lstm['data'].shape[0]
        self.n_val_lstm = self.val_set_lstm['data'].shape[0]

    def get_train_data(self):
        return DataLoader(TensorDataset(self.train_data), batch_size=self.config['batch_size'], shuffle=True)

    def get_val_data(self):
        return DataLoader(TensorDataset(self.val_data), batch_size=self.config['batch_size'], shuffle=False)

In [5]:
def visualize_prediction(model, sequence, config, index):
    with torch.no_grad():
        # Encode sequence
        encoded_sequence = model.generate_embeddings(sequence.unsqueeze(0))
        
        # Predict next embedding
        predicted_embedding = model.lstm(torch.from_numpy(encoded_sequence[:, :-1]).float().to(model.device))
        
        # Decode original and predicted sequence
        original_decoded = model.vae.decode(torch.from_numpy(encoded_sequence).float().to(model.device))
        predicted_decoded = model.vae.decode(predicted_embedding[:, -1])

    # Plot results
    fig, axs = plt.subplots(config['n_channel'], 1, figsize=(15, 5*config['n_channel']))
    if config['n_channel'] == 1:
        axs = [axs]
    
    for ch in range(config['n_channel']):
        axs[ch].plot(sequence[:, :, ch].flatten().numpy(), label='Original')
        axs[ch].plot(original_decoded[:, :, ch].cpu().flatten().numpy(), label='VAE Reconstruction')
        axs[ch].plot(np.arange(config['l_win']*(config['l_seq']-1), config['l_win']*config['l_seq']),
                     predicted_decoded[0, :, ch].cpu().numpy(), label='LSTM Prediction')
        axs[ch].set_title(f'Channel {ch+1}')
        axs[ch].legend()

    plt.tight_layout()
    plt.savefig(os.path.join(config['result_dir'], f'prediction_{index}.png'))
    plt.close()

In [43]:
def main():
    global train_configurations
    config = train_configurations    

    # Create the experiments dirs
    create_dirs([config['result_dir'], config['checkpoint_dir'], config['checkpoint_dir_lstm']])

    # Set device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Create your data generator
    data = DataGenerator()

    # Create VAE-LSTM model
    model = VAE_LSTM_Model(config)

    # Train VAE
    if config['TRAIN_VAE']:
        print('Train VAE Process...')
        train_loader = data.get_train_data()
        val_loader = data.get_val_data()
        
        for epoch in range(config['num_epochs_vae']):
            model.vae.train()
            for batch in train_loader:
                x = batch[0].to(device)
                # The shape of x is: 
                print(f'x shape: {x.shape}')
                loss = model.train_vae(x)
            
            # Validation
            model.vae.eval()
            val_loss = 0
            with torch.no_grad():
                for batch in val_loader:
                    x = batch[0].to(device)
                    recon_x, mu, log_var = model.vae(x)
                    val_loss += model.vae_loss(recon_x, x, mu, log_var).item()
            val_loss /= len(val_loader)
            
            print(f'VAE Epoch: {epoch}, Train Loss: {loss:.4f}, Val Loss: {val_loss:.4f}')

        # Save VAE model
        torch.save(model.vae.state_dict(), os.path.join(config['checkpoint_dir'], 'vae_model.pth'))

    # Train LSTM
    if config['TRAIN_LSTM']:
        print('Train LSTM Process...')
        # Generate embeddings
        train_embeddings = model.generate_embeddings(data.train_set_lstm['data'])
        val_embeddings = model.generate_embeddings(data.val_set_lstm['data'])

        # Prepare LSTM data
        x_train = train_embeddings[:, :-1]
        y_train = train_embeddings[:, 1:]
        x_val = val_embeddings[:, :-1]
        y_val = val_embeddings[:, 1:]

        train_loader = DataLoader(TensorDataset(torch.from_numpy(x_train), torch.from_numpy(y_train)),
                                  batch_size=config['batch_size_lstm'], shuffle=True)
        val_loader = DataLoader(TensorDataset(torch.from_numpy(x_val), torch.from_numpy(y_val)),
                                batch_size=config['batch_size_lstm'], shuffle=False)

        for epoch in range(config['num_epochs_lstm']):
            model.lstm.train()
            for batch_x, batch_y in train_loader:
                batch_x, batch_y = batch_x.to(device), batch_y.to(device)
                loss = model.train_lstm_step(batch_x, batch_y)

            # Validation
            model.lstm.eval()
            val_loss = 0
            with torch.no_grad():
                for batch_x, batch_y in val_loader:
                    batch_x, batch_y = batch_x.to(device), batch_y.to(device)
                    output = model.lstm(batch_x)
                    val_loss += F.mse_loss(output, batch_y).item()
            val_loss /= len(val_loader)

            print(f'LSTM Epoch: {epoch}, Train Loss: {loss:.4f}, Val Loss: {val_loss:.4f}')

        # Save LSTM model
        torch.save(model.lstm.state_dict(), os.path.join(config['checkpoint_dir_lstm'], 'lstm_model.pth'))

    # Visualize results
    #for i in range(10):
    #    visualize_prediction(model, data.val_set_lstm['data'][i], config, i)

In [44]:
if __name__ == '__main__':
    main()

KeyError: 'result_dir'

In [1]:
from arch_skip import VAE_LSTM_Model

train_configurations = {
    "window_dim": 32, # CAUTION: changing this value requires changing the model architecture
    "hidden_dim": 512,
    "latent_dim": 6,
    "learning_rate_vae": 1e-3,
}
modelx = VAE_LSTM_Model(train_configurations)
from torchsummary import summary
summary(modelx.vae, (1, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1               [-1, 32, 16]             160
            Conv1d-2                [-1, 64, 8]           6,208
            Conv1d-3                [-1, 64, 8]           2,112
            Conv1d-4               [-1, 128, 4]          24,704
            Conv1d-5               [-1, 128, 4]           8,320
            Conv1d-6               [-1, 256, 2]          98,560
            Conv1d-7               [-1, 256, 2]          33,024
            Conv1d-8               [-1, 512, 1]         393,728
            Conv1d-9               [-1, 512, 1]         131,584
           Linear-10                    [-1, 6]           3,078
           Linear-11                    [-1, 6]           3,078
           Linear-12                  [-1, 512]           3,584
  ConvTranspose1d-13               [-1, 256, 2]         524,544
  ConvTranspose1d-14               [-1,

In [ ]:
print(modelx.vae)

VAE(
  (encoder): Sequential(
    (0): Conv1d(1, 32, kernel_size=(3,), stride=(2,), padding=(1,))
    (1): LeakyReLU(negative_slope=0.01)
    (2): Conv1d(32, 64, kernel_size=(3,), stride=(2,), padding=(1,))
    (3): LeakyReLU(negative_slope=0.01)
    (4): Conv1d(64, 128, kernel_size=(3,), stride=(2,), padding=(1,))
    (5): LeakyReLU(negative_slope=0.01)
    (6): Conv1d(128, 512, kernel_size=(4,), stride=(1,))
    (7): LeakyReLU(negative_slope=0.01)
  )
  (decoder): Sequential(
    (0): ConvTranspose1d(512, 256, kernel_size=(3,), stride=(2,), padding=(1,), output_padding=(1,))
    (1): LeakyReLU(negative_slope=0.01)
    (2): ConvTranspose1d(256, 128, kernel_size=(3,), stride=(2,), padding=(1,), output_padding=(1,))
    (3): LeakyReLU(negative_slope=0.01)
    (4): ConvTranspose1d(128, 64, kernel_size=(3,), stride=(2,), padding=(1,), output_padding=(1,))
    (5): LeakyReLU(negative_slope=0.01)
    (6): ConvTranspose1d(64, 1, kernel_size=(3,), stride=(2,), padding=(1,), output_padding=(1,